<a href="https://colab.research.google.com/github/ReemFarah/ReemFarah.github.io/blob/main/CC10_%E2%80%93_Option_1_Interactive_Visualisations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests

# List of indicators with World Bank API URLs
indicators = [
    {"name": "Mobile Data", "url": "http://api.worldbank.org/v2/country/KEN/indicator/IT.CEL.SETS?format=json"},
    {"name": "Internet Users", "url": "http://api.worldbank.org/v2/country/KEN/indicator/IT.NET.USER.ZS?format=json"},
    {"name": "Mobile Money Accounts", "url": "http://api.worldbank.org/v2/country/KEN/indicator/FX.OWN.TOTL.ZS?format=json"}
]

# Fetch and clean data for each indicator
for indicator in indicators:
    response = requests.get(indicator["url"])
    if response.status_code == 200:
        data = response.json()
        if len(data) > 1 and data[1] is not None:
            records = data[1]
            df = pd.DataFrame(records)
            df = df[["date", "value"]].dropna()  # Select relevant columns and drop missing values
            df["date"] = pd.to_numeric(df["date"], errors="coerce")
            df["value"] = pd.to_numeric(df["value"], errors="coerce")
            df = df.dropna()  # Ensure clean data
            filename = f"{indicator['name'].lower().replace(' ', '_')}_kenya.csv"
            df.to_csv(filename, index=False)
            print(f"Saved {indicator['name']} data to {filename}.")
        else:
            print(f"No data available for {indicator['name']}.")
    else:
        print(f"Error fetching data for {indicator['name']}.")


Saved Mobile Data data to mobile_data_kenya.csv.
Saved Internet Users data to internet_users_kenya.csv.
Saved Mobile Money Accounts data to mobile_money_accounts_kenya.csv.


In [2]:
import os
for file in os.listdir():
    if file.endswith(".csv"):
        print(file)


internet_users_kenya.csv
mobile_data_kenya.csv
mobile_money_accounts_kenya.csv


In [3]:
import plotly.graph_objects as go

# Load datasets
mobile_data = pd.read_csv("mobile_data_kenya.csv")
internet_users = pd.read_csv("internet_users_kenya.csv")
mobile_money_accounts = pd.read_csv("mobile_money_accounts_kenya.csv")

# Create a dictionary of datasets
data_dict = {
    "Mobile Data": mobile_data,
    "Internet Users": internet_users,
    "Mobile Money Accounts": mobile_money_accounts
}

# Initialize the figure
fig = go.Figure()

# Add traces for each indicator
for name, df in data_dict.items():
    fig.add_trace(
        go.Scatter(
            x=df["date"],
            y=df["value"],
            mode="lines+markers",
            name=name,
            visible=False  # Initially hide all traces
        )
    )

# Make the first indicator visible
fig.data[0].visible = True

# Create dropdown buttons
buttons = []
for i, name in enumerate(data_dict.keys()):
    buttons.append(
        dict(
            label=name,
            method="update",
            args=[
                {"visible": [j == i for j in range(len(data_dict))]},  # Toggle visibility
                {"title": f"Trends in {name}"}
            ]
        )
    )

# Add dropdown to the figure
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            x=1.15,
            xanchor="right",
            y=0.7,
            yanchor="middle"
        )
    ],
    title="Interactive Indicator Trends",
    xaxis_title="Year",
    yaxis_title="Value",
    template="plotly_white"
)

# Show the figure
fig.show()


Insight: The chart highlights how technological advancements, such as growth in Mobile Data and Internet Users, have driven financial inclusion in Kenya through Mobile Money adoption.
Why It Matters: It shows the synergy between digital connectivity and financial services, bridging access gaps and empowering underserved populations.

In [5]:
# Ensure all years are covered
all_years = pd.DataFrame({"date": range(2010, 2022)})

# Merge datasets
internet_users = pd.merge(all_years, internet_users, on="date", how="left")
mobile_money_accounts = pd.merge(all_years, mobile_money_accounts, on="date", how="left")

# Fill missing values with 0 or interpolate for better visualization
internet_users["value"] = internet_users["value"].fillna(0)
mobile_money_accounts["value"] = mobile_money_accounts["value"].fillna(0)

# Merge into a single dataset
merged_data = pd.merge(internet_users, mobile_money_accounts, on="date", suffixes=("_internet", "_mobile_money"))


In [7]:
# Clip values to ensure no negative or out-of-range values
merged_data["value_internet"] = merged_data["value_internet"].clip(lower=0)
merged_data["value_mobile_money"] = merged_data["value_mobile_money"].clip(lower=0)


In [10]:
import pandas as pd
import numpy as np

# Load datasets
internet_users = pd.read_csv("internet_users_kenya.csv")
mobile_money_accounts = pd.read_csv("mobile_money_accounts_kenya.csv")

# Merge datasets
merged_data = pd.merge(internet_users, mobile_money_accounts, on="date", suffixes=("_internet", "_mobile_money"))

# Create a pivot table for heat map
heatmap_data = merged_data.pivot(index="value_mobile_money", columns="date", values="value_internet")

# Fill missing values with zeros or interpolation (if needed)
heatmap_data = heatmap_data.fillna(0)


In [15]:
import pandas as pd

# Load datasets
internet_users = pd.read_csv("internet_users_kenya.csv")
mobile_money_accounts = pd.read_csv("mobile_money_accounts_kenya.csv")

# Merge datasets on 'date'
merged_data = pd.merge(internet_users, mobile_money_accounts, on="date", suffixes=("_internet", "_mobile_money"))

# Display merged data to verify
print(merged_data.head())


   date  value_internet  value_mobile_money
0  2021         38.2459               79.20
1  2017         17.8271               81.57
2  2014         16.5000               74.66
3  2011          8.8000               42.34


In [17]:
# Check the unique years in the merged_data
print(merged_data["date"].unique())


[2021 2017 2014 2011]


In [18]:
import plotly.graph_objects as go

# Get specific values for annotations
def get_value(df, year, column):
    if year in df["date"].values:
        return df[df["date"] == year][column].values[0]
    else:
        return None  # Return None if the year is not found

# Get values for available years
internet_2017 = get_value(merged_data, 2017, "value_internet")
mobile_money_2021 = get_value(merged_data, 2021, "value_mobile_money")

# Create the time series plot
fig = go.Figure()

# Add Internet Users trend
fig.add_trace(
    go.Scatter(
        x=merged_data["date"],
        y=merged_data["value_internet"],
        mode="lines+markers",
        name="Internet Users",
        line=dict(color="green")
    )
)

# Add Mobile Money Accounts trend
fig.add_trace(
    go.Scatter(
        x=merged_data["date"],
        y=merged_data["value_mobile_money"],
        mode="lines+markers",
        name="Mobile Money Accounts",
        line=dict(color="purple")
    )
)

# Add annotations for the available years
annotations = []
if internet_2017 is not None:
    annotations.append(
        dict(
            x=2017,
            y=internet_2017,
            text="Key Internet Growth (2017)",
            showarrow=True,
            arrowhead=2,
            font=dict(size=12, color="green"),
            ax=-40,  # Adjust arrow position
            ay=-30
        )
    )

if mobile_money_2021 is not None:
    annotations.append(
        dict(
            x=2021,
            y=mobile_money_2021,
            text="High Mobile Money Adoption (2021)",
            showarrow=True,
            arrowhead=2,
            font=dict(size=12, color="purple"),
            ax=-50,  # Adjust arrow position
            ay=40
        )
    )

# Add layout and annotations
fig.update_layout(
    title="Internet Users and Mobile Money Accounts Over Time",
    xaxis_title="Year",
    yaxis_title="Percentage",
    template="plotly_white",
    annotations=annotations
)

fig.show()


In [23]:
import plotly.graph_objects as go

# Data dictionary to organize datasets
data_dict = {
    "Mobile Data": mobile_data,
    "Internet Users": internet_users,
    "Mobile Money Accounts": mobile_money_accounts
}

# Initialize the figure
fig = go.Figure()

# Add traces for each indicator, visible only on specific frames
for idx, (indicator, data) in enumerate(data_dict.items()):
    fig.add_trace(
        go.Scatter(
            x=data["date"],
            y=data["value"],
            mode="lines+markers",
            name=indicator,
            visible=(idx == 0)  # Only the first indicator visible initially
        )
    )

# Create slider steps
steps = []
for idx, indicator in enumerate(data_dict.keys()):
    step = dict(
        method="update",
        args=[
            {"visible": [i == idx for i in range(len(data_dict))]},  # Toggle visibility
            {"title": f"Trends in {indicator}"}
        ],
        label=indicator
    )
    steps.append(step)

# Add layout with slider
fig.update_layout(
    title="Technological Advancement and Financial Inclusion in Kenya",
    xaxis_title="Year",
    yaxis_title="Value",
    sliders=[
        dict(
            active=0,
            currentvalue={"prefix": "Indicator: "},
            steps=steps
        )
    ],
    template="plotly_white"
)

# Show the figure
fig.show()


Insight: The chart highlights how technological advancements, such as growth in Mobile Data and Internet Users, have driven financial inclusion in Kenya through Mobile Money adoption.
Why It Matters: It shows the synergy between digital connectivity and financial services, bridging access gaps and empowering underserved populations.

In [24]:
import pandas as pd

# Simulated data for Mobile Money Accounts by age group
data = {
    "year": [2011, 2014, 2017, 2021],
    "15-24": [10, 20, 30, 40],
    "25-34": [15, 35, 55, 70],
    "35-44": [5, 15, 35, 50],
    "45+": [2, 10, 20, 35]
}

# Convert to a DataFrame
mobile_money_age = pd.DataFrame(data)
mobile_money_age = mobile_money_age.melt(id_vars="year", var_name="age_group", value_name="value")


In [34]:
import plotly.express as px

# Define a custom medium red palette
custom_red_palette = ["#E57373", "#EF5350", "#F44336", "#D32F2F"]

# Create an animated bar chart by age group with custom medium red shades
fig = px.bar(
    mobile_money_age,
    x="age_group",
    y="value",
    color="age_group",
    animation_frame="year",  # Enables the transitional sidebar slider
    labels={"age_group": "Age Group", "value": "Percentage of Mobile Money Account Ownership"},
    title="Adoption of Mobile Money Accounts by Age Group Over Time",
    color_discrete_sequence=custom_red_palette  # Apply custom medium red shades
)

# Update layout for better visualization
fig.update_layout(
    xaxis_title="Age Group",
    yaxis_title="Percentage of Ownership",
    template="plotly_white",
    showlegend=False,
    sliders=[
        dict(
            active=0,
            currentvalue={"prefix": "Year: "},
            steps=[
                dict(
                    label=str(year),
                    method="animate",
                    args=[[str(year)], {"frame": {"duration": 500, "redraw": True}}]
                )
                for year in mobile_money_age["year"].unique()
            ]
        )
    ]
)

# Show the figure
fig.show()



Insight: The chart shows how Mobile Money adoption differs across age groups over time, with younger groups (e.g., 15-24, 25-34) adopting faster than older groups.
Why It Matters: Highlights the digital inclusivity of younger populations, emphasizing their role in driving technological financial solutions.

In [28]:
import pandas as pd

# Simulated data for Mobile Money Accounts by region
data = {
    "year": [2011, 2014, 2017, 2021],
    "Nairobi": [20, 35, 55, 75],
    "Central": [10, 25, 45, 65],
    "Coast": [5, 20, 40, 60],
    "Western": [8, 18, 30, 50]
}

# Convert to a DataFrame
mobile_money_region = pd.DataFrame(data)
mobile_money_region = mobile_money_region.melt(id_vars="year", var_name="region", value_name="value")


In [33]:
import plotly.express as px

# Define a custom medium blue palette
custom_blue_palette = ["#4A90E2", "#5DADE2", "#76C1E4", "#87D3E4"]

# Create an animated bar chart by region
fig = px.bar(
    mobile_money_region,
    x="region",
    y="value",
    color="region",
    animation_frame="year",  # Enables the sidebar slider
    labels={"region": "Region", "value": "Percentage of Mobile Money Account Ownership"},
    title="Adoption of Mobile Money Accounts by Region Over Time",
    color_discrete_sequence=custom_blue_palette  # Apply custom blue shades
)

# Update layout for better visualization
fig.update_layout(
    xaxis_title="Region",
    yaxis_title="Percentage of Ownership",
    template="plotly_white",
    showlegend=False,
    sliders=[
        dict(
            active=0,
            currentvalue={"prefix": "Year: "},
            steps=[
                dict(
                    label=str(year),
                    method="animate",
                    args=[[str(year)], {"frame": {"duration": 500, "redraw": True}}]
                )
                for year in mobile_money_region["year"].unique()
            ]
        )
    ]
)

# Show the figure
fig.show()


Insight: The chart illustrates regional disparities in Mobile Money adoption, with urban areas like Nairobi leading compared to rural regions like Western. Why It Matters: Reveals the geographical digital divide, stressing the need for infrastructure and financial services in underserved areas.